<h1> Modelling <span class="tocSkip"></span></h1>

Having ran a baseline model, and having conducted further preprocessing, I would now like to see whether I can replicate similar steps to Story et al. and see how different preprocessing steps affect the performance of models.

I will begin by replicating their steps, but just training a single classifier.  Then I will evaluate the results. This will be section 1 of this notebook.

Then, in section 2, having demonstrated the steps, I will build functions for training models to use to train multiple classifiers.

This will give results for all classifiers that I can then discuss.

Finally I can run my most promising models on the test set.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Crafted-Features-and-Sentence-Filtering" data-toc-modified-id="Crafted-Features-and-Sentence-Filtering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Crafted Features and Sentence Filtering</a></span></li><li><span><a href="#Section-1" data-toc-modified-id="Section-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Section 1</a></span><ul class="toc-item"><li><span><a href="#Loading-data-and-splitting-into-train/validate/test" data-toc-modified-id="Loading-data-and-splitting-into-train/validate/test-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Loading data and splitting into train/validate/test</a></span></li></ul></li><li><span><a href="#Step-1:-select-classifier:-&quot;1st_party&quot;" data-toc-modified-id="Step-1:-select-classifier:-&quot;1st_party&quot;-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Step 1: select classifier: "1st_party"</a></span></li><li><span><a href="#Step-2:-Separate-into-X-and-y" data-toc-modified-id="Step-2:-Separate-into-X-and-y-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 2: Separate into X and y</a></span></li><li><span><a href="#Step-3:-Set-up-filter-for-sentence-filtering-to-use-in-Pipeline" data-toc-modified-id="Step-3:-Set-up-filter-for-sentence-filtering-to-use-in-Pipeline-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 3: Set up filter for sentence filtering to use in Pipeline</a></span></li><li><span><a href="#Step-4:-Set-up-for-tf-idf-matrix-to-use-in-Pipeline" data-toc-modified-id="Step-4:-Set-up-for-tf-idf-matrix-to-use-in-Pipeline-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 4: Set up for tf-idf matrix to use in Pipeline</a></span></li><li><span><a href="#Step-5:-5-fold-CV-Grid-Search" data-toc-modified-id="Step-5:-5-fold-CV-Grid-Search-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 5: 5-fold CV Grid Search</a></span></li><li><span><a href="#Evaluation-on-Train-dataset" data-toc-modified-id="Evaluation-on-Train-dataset-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Evaluation on Train dataset</a></span></li><li><span><a href="#Evaluation-on-Validation-dataset" data-toc-modified-id="Evaluation-on-Validation-dataset-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Evaluation on Validation dataset</a></span></li><li><span><a href="#Section-2:-modelling-pipeline" data-toc-modified-id="Section-2:-modelling-pipeline-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Section 2: modelling pipeline</a></span><ul class="toc-item"><li><span><a href="#Requirements-for-modelling-pipeline" data-toc-modified-id="Requirements-for-modelling-pipeline-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Requirements for modelling pipeline</a></span></li></ul></li><li><span><a href="#Old-from-here:" data-toc-modified-id="Old-from-here:-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Old from here:</a></span></li><li><span><a href="#Run-all-classifiers-through-the-pipeline" data-toc-modified-id="Run-all-classifiers-through-the-pipeline-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Run all classifiers through the pipeline</a></span></li><li><span><a href="#Creating-model-evaluation-table" data-toc-modified-id="Creating-model-evaluation-table-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Creating model evaluation table</a></span></li><li><span><a href="#Discussion-of-low-F1-scores" data-toc-modified-id="Discussion-of-low-F1-scores-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Discussion of low F1 scores</a></span><ul class="toc-item"><li><span><a href="#A-wide-range-of-scores" data-toc-modified-id="A-wide-range-of-scores-14.1"><span class="toc-item-num">14.1&nbsp;&nbsp;</span>A wide range of scores</a></span></li><li><span><a href="#Neg-F1-tends-to-be-higher-than-Pos-F1" data-toc-modified-id="Neg-F1-tends-to-be-higher-than-Pos-F1-14.2"><span class="toc-item-num">14.2&nbsp;&nbsp;</span>Neg F1 tends to be higher than Pos F1</a></span></li><li><span><a href="#Compare-with-results-from-the-paper" data-toc-modified-id="Compare-with-results-from-the-paper-14.3"><span class="toc-item-num">14.3&nbsp;&nbsp;</span>Compare with results from the paper</a></span></li><li><span><a href="#Matching-scores-for-SSO-and-FacebookSSO" data-toc-modified-id="Matching-scores-for-SSO-and-FacebookSSO-14.4"><span class="toc-item-num">14.4&nbsp;&nbsp;</span>Matching scores for SSO and FacebookSSO</a></span></li></ul></li><li><span><a href="#Final-model-performance-on-test-set" data-toc-modified-id="Final-model-performance-on-test-set-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Final model performance on test set</a></span></li><li><span><a href="#Further-work" data-toc-modified-id="Further-work-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Further work</a></span></li><li><span><a href="#OLD:-TF-IDF-matrix-(old-content)" data-toc-modified-id="OLD:-TF-IDF-matrix-(old-content)-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>OLD: TF-IDF matrix (old content)</a></span><ul class="toc-item"><li><span><a href="#Create-y-(old-content)" data-toc-modified-id="Create-y-(old-content)-17.1"><span class="toc-item-num">17.1&nbsp;&nbsp;</span>Create y (old content)</a></span></li></ul></li><li><span><a href="#Step-2:-apply-Sentence-Filtering-(old-content)" data-toc-modified-id="Step-2:-apply-Sentence-Filtering-(old-content)-18"><span class="toc-item-num">18&nbsp;&nbsp;</span>Step 2: apply Sentence Filtering (old content)</a></span></li><li><span><a href="#Evaluation-on-Validation-dataset-(Old-content)" data-toc-modified-id="Evaluation-on-Validation-dataset-(Old-content)-19"><span class="toc-item-num">19&nbsp;&nbsp;</span>Evaluation on Validation dataset (Old content)</a></span></li></ul></div>

The steps in this notebook are:

<font size= "4"> **Section 1** </font>

- Load data and split into Train/Validate/Test

    - For demonstrative purposes I will begin by running through the modelling steps for just a single classifier: 1st Party
    

- Separating into X and y
  - X, the input data to the model, is a union of the crafted features columns and a tf-idf matrix


- 5-fold cross-validation grid search
  - Over Sentence Filtering, whether to include bigrams in the tf-idf matrix and the hyperparameters for SVM model and  as performed by Story et al.
  - Additionally I opt to search over a default logistic regression classifier for comparison


- Evaluation on train dataset


- Evaluation on validation set and review.

<font size= "4"> **Section 2** </font>

Having demonstrated the steps for model construction for one classifier, I move to create a pipeline to train multiple classifiers.

The full pipeline consists of a series of functions that allows me to train a classifier for each of the 18 targets of interest. The steps are:

- Listing the requirements for the pipeline

- Functions for the pipeline
  - These correspond to the steps executed above for ‘1st party’

- Display of all model scores

- Discussion of some zero scores

- Further discussion

- Final model performance

# Crafted Features and Sentence Filtering

I have discussed this in previous notebooks but will include it here for completion.

To try to improve classifier performance, Story et al. apply a preprocessing technique that they call Sentence Filtering. This involves filtering the data to only train a classifier on segments that contain a relevant feature for the target.  For example a short segment about location data wouldn't be used to train an email classifier because it wouldn't contain keywords such as "Email" or "Contact".

They do not explicitly specify that these features are equivalent to the crafted features but I am inferring that they are.

The effect of applying sentence filtering is that instead of a model being trained on all the data and there being class imbalance in favour of the negative class, a much greater proportion of the training data features the target being classified. The effect is similar to downsampling.

This will be a moderate approximation for a replication of most of their work. The main missing element will be better text pre-processing to get better results from the crafted features and sentence filtering.

In [1]:
import pandas as pd
from pandas import json_normalize
import yaml
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from scipy import stats
from scipy.stats import norm

import sys
import time
from collections import defaultdict
from collections import Counter

import priv_policy_manipulation_functions as priv_pol_funcs

# pre-processing
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack

# modelling
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# modelling pipeline
from tempfile import mkdtemp
# from sklearn.pipeline import Pipeline # not required since using imbalanced learn
from imblearn.pipeline import Pipeline 
    # Using the pipeline from the imbalanced learn library since it allows for sampling
from imblearn import FunctionSampler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

# modelling evaluation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# Section 1
## Loading data and splitting into train/validate/test

**Creating Train, Validate and Test dataframes to use.** <br/> Story et al. already randomly split them into training, validation and test. They do not explain that this split was stratified by any classifier.  The proportions of the splits that they use seem standard for common data science practice – above a 25% test split.  

I am suspicious that there is not enough data in this domain to train some classifiers so I would have preferred this proportion to be lower, closer to 25% or perhaps slightly below.  (But I think this would have been difficult for Story et al. to know in advance and they may have had an idea already since they were building on earlier work.)

In [32]:
# Load the data
df_for_pipelining = pd.read_pickle("objects/crafted_features_df.pkl")

# Create separate dataframes for each group
df_for_pipelining_train = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'TRAINING' ].copy()
df_for_pipelining_val = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'VALIDATION' ].copy()
df_for_pipelining_test = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'TEST' ].copy()

# now that I have used the 'policy type' column for referring to train/validate/test, 
# I can delete that column along with other unneccesary columns.
for dataframe in [df_for_pipelining_train, df_for_pipelining_val, df_for_pipelining_test]:
    dataframe.drop(columns=['source_policy_number', 'policy_type', 'contains_synthetic',
           'policy_segment_id', 'annotations', 'sentences'], inplace=True)
    dataframe.reset_index(inplace=True, drop=True)

Sanity Check – Seeing the shapes of the dataframe – number of observations and confirming the number of columns

In [3]:
print(df_for_pipelining_train.shape)
print(df_for_pipelining_val.shape)
print(df_for_pipelining_test.shape)

(8068, 511)
(2651, 511)
(4824, 511)


In [4]:
# annotation features to use for sentence filtering later
clean_annotation_features = pd.read_pickle("objects/clean_annotation_features.pkl")

# Step 1: select classifier: "1st_party"

Let's start with 1st Party as an example.

In [5]:
classifier = "1st_party"

# Step 2: Separate into X and y



In [9]:
# Separate into X and y
# the crafted features columns happen to be all those after and including 'contact info', so I 
# use every column after and including the first crafted feature, which happens to be 'contact info'
X = pd.concat(
    [df_for_pipelining_train['segment_text'], 
     df_for_pipelining_train.loc[:,'contact info':]],
    axis=1
).copy()

print("Should be left with the 476 different crafted features (CFs), plus one segment text column.")
print(f"X shape is: {X.shape}")

y = df_for_pipelining_train[classifier]

Should be left with the 476 different crafted features (CFs), plus one segment text column.
X shape is: (8068, 477)


Now we have our input data and target ready to pass into our model.

# Step 3: Set up filter for sentence filtering to use in Pipeline

Retrieving the crafted features for 1st Party to use for sentence filtering:

In [10]:
# filtering the table to get the list object from the same row that lists the classifier
classifier_features = clean_annotation_features[ clean_annotation_features['annotation'] == classifier ]     \
                        .reset_index().at[0,'features']

classifier_features

['we', 'you', 'us', 'our', 'the app', 'the software']

Filter for the dataframe to get rows where any of those features is 1:

In [13]:
# true/false boolean series for sentence filtering:
sf_filter = ((X[classifier_features] > 0)\
                 .sum(axis=1) > 0 )

Filter for the dataframe to conduct balanced downsize filtering. This is a feature of my sentence_filtering function to filter less stringently if there is not enough data left after using the sf_filter.

In [14]:
# true/false boolean series for balanced downsizing filter:
positive_rows = (y == 1)
negative_rows = (y == 0)
balanced_downsize_filter = (
    positive_rows |
    negative_rows.where(negative_rows == True).dropna().sample(n=positive_rows.sum(), replace=False)
)

These arguments will be passed into the sentence filtering function during the pipeline

In [15]:
sf_kw_args = {'df_filter': sf_filter, 'sf_filter': sf_filter, 'balanced_downsize_filter': balanced_downsize_filter}

# Step 4: Set up for tf-idf matrix to use in Pipeline

Using 'column transformer' in the pipeline, and by creating the tf-idf objects in advance, we don't need to manually create the tf-idf matrix, fit it to the data, or transform the data with it, or combine the resulting sparse matrix with the crafted features dataframe.

In [12]:
# tf-idf functions to use
tfidf_unigrams = TfidfVectorizer(ngram_range=(1,1), stop_words='english', binary=True)
tfidf_withbigrams = TfidfVectorizer(ngram_range=(1,2), stop_words='english', binary=True)

# Create the column transformations list with columns to apply to (for the column transformer)
col_transform_unigrams = [('unigrams_only', tfidf_unigrams, 'segment_text')]
col_transform_withbigrams = [('with_bigrams', tfidf_withbigrams, 'segment_text')]

# Step 5: 5-fold CV Grid Search

There are multiple things going on here that are worth explaining:
- 5-fold CV (cross-validation)
- Model of choice: SVM
- Hyperparameter Grid Search
- A note on scoring

**5-fold Cross Validation**

The technique involves partitioning the dataset into five folds, each of which is used in turn as a test set while the remaining folds are used as training data for a model. This process is repeated a number of times, with each fold being used as a test set once. The average performance across all test sets is then used to judge the model's performance.

This means that each different model being tested in the grid search is trained and tested 5 times. This approach can be computationally intensive, but it often results in a more accurate estimate of the model's true performance, because each model is being tested on data that it hasn't seen.

**Model of Choice: SVM (Support Vector Machines)**

Of all possible machine learning models, Story et al. exclusively mention SVM. I presume that other models were not discussed because:
- they were building on earlier work informing them of how to find success with SVM in this domain, or
- neglected to explain all the other model searching they conducted

SVM is a supervised machine learning algorithm that can be used for both classification and regression tasks. In the n-dimensional space of training datapoints, the SVM algorithm finds the hyperplane that maximizes the margin between the data of the two classes (i.e. the decision boundary). The margin is defined as the distance between the hyperplane and the closest data points.  The closest data points to the margin are used to construct the hyperplane and are thus called the 'support vectors'. This image helps demonstrate the idea. The Kernel is a transformation to help separate the classes.

<img src="https://www.analytixlabs.co.in/blog/wp-content/uploads/2021/07/Blog-08-1-1536x832.jpg" alt="An example of SVM. Image from analytixlabs" width=700/>

*Image source: analytixlabs*

The SVM algorithm has a number of advantages, including its ability to handle non-linear decision boundaries, its robustness to overfitting, and its efficiency in high-dimensional spaces such as with text data.

In general the time complexity for support vector machines is between O(m * n^2) and O(m * n^3), where m is the number of features and n is the number of training examples, but it is more complex than this and can be specific to different kernels.  I find greatly increasing training time with more observations. But with this small dataset, given the width of the data, the training times are very manageable.

I recommend [Sci-kit Learn's page](https://scikit-learn.org/stable/modules/svm.html) to learn more.

**SVM: Which hyperparameters can be tuned?**

Story et al. state: "we use scikit-learn’s SVC implementation (scikit-learn developers 2016b). We train those with a linear kernel (kernel=’linear’), balanced class weights (class weight=’balanced’), and a grid search with five-fold cross-validation over the penalty (C=[0.1, 1, 10]) and gamma (gamma=[0.001, 0.01, 0.1]) parameters.

- The Kernel function for transformation, e.g. rbf, polynomial, linear. Having decided that Linear is suitable for this problem, Story et al. only consider a linear kernel, which is known to perform well on text data.
- C. Decreasing C corresponds to more regularization. The C parameter adds a penalty for each incorrect classification, so a smaller C value results in a decision boundary with a larger margin, but may also result in a higher number of misclassifications.
- Gamma. This is not relevant for linear kernels so I'm unsure why they have included it. (See the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) ). It is possible that it was left there as an error after the researchers tried different kernels. The code runs either way.  Although I want to include it in case the researchers had some reason, to save computation time I will not include it in my grid search.

**Grid Search**
To find the best model for each target, we want to find the combination of pre-processing steps and hyperparameters that gives the best results. So for each classifier, we train and evaluate models for each different combination of pre-processing steps and hyperparameters. Effectively, this 'grid' of steps and hyperparameters is searched over to find the best model.

In this case, we are searching over:

- Sentence filtering or not
- tfidf with only unigrams or unigrams and bigrams
- SVC with C=[0.1, 1, 10], OR
- if not SVC, logistic regression.

**Scoring**

I am electing to assess each model's performance by the F1 score (harmonic mean of precision and recall). The F1 metric is an important metric for Story et al.'s work so this will help me compare.

In [25]:
cachedir = mkdtemp() # Memory dump to help with processing

# In the pipeline_sequences list, the syntax requires that each step is named and some value is provided,
# but that value will be changed as each different option is looped through in the parameter grid.
pipeline_sequences = [
    ('sentence_filtering', FunctionSampler(func=priv_pol_funcs.sentence_filtering, validate=False, kw_args=sf_kw_args)),
    ('tfidf', ColumnTransformer(col_transform_withbigrams, remainder='passthrough')), 
    ('model', LogisticRegression(random_state=1, max_iter=1000))
]

pipe = Pipeline(pipeline_sequences, memory = cachedir)

param_grid = [
    {
        'model': [LogisticRegression(random_state=1, max_iter=1000)],
        'sentence_filtering': [FunctionSampler(func=priv_pol_funcs.sentence_filtering, validate=False, kw_args=sf_kw_args), 
                               None],
        'tfidf': [ColumnTransformer(col_transform_withbigrams, remainder='passthrough'),
                  ColumnTransformer(col_transform_unigrams, remainder='passthrough')]
    },
    {
        'model': [SVC(kernel='linear', class_weight='balanced', random_state=1)],
        'model__C': [0.1, 1, 10],
        'sentence_filtering': [FunctionSampler(func=priv_pol_funcs.sentence_filtering, validate=False, kw_args=sf_kw_args), 
                               None],
        'tfidf': [ColumnTransformer(col_transform_withbigrams, remainder='passthrough'),
                  ColumnTransformer(col_transform_unigrams, remainder='passthrough')]
    }
]

# Create grid search object
grid_search_object = GridSearchCV(estimator=pipe, param_grid = param_grid, cv = 5, verbose=1, n_jobs=-1, scoring='f1')


In [26]:
%%time
fitted_search = grid_search_object.fit(X, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.29s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.25s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib'

CPU times: user 11.2 s, sys: 309 ms, total: 11.5 s
Wall time: 1min 28s


# Evaluation on Train dataset

In [28]:
classifier_prediction = fitted_search.predict(X)
print(classification_report(y, classifier_prediction))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      6235
           1       0.93      1.00      0.96      1833

    accuracy                           0.98      8068
   macro avg       0.97      0.99      0.98      8068
weighted avg       0.98      0.98      0.98      8068



These seem like healthy scores across the board but they are all quite high so there is a risk of overfitting.  I will discuss the results once evaluating the score on the validation set, which allows me to compare to Story et al.

# Evaluation on Validation dataset

Loading the X and y data

In [34]:
Xval = pd.concat(
    [df_for_pipelining_val['segment_text'], 
     df_for_pipelining_val.loc[:,'contact info':]],
    axis=1
).copy()

yval = df_for_pipelining_val[classifier]

**Storing the model results** 

I create a series to store the results for each classifier I could make.

For each model, I will create a list with 3 values: the fitted search object, the y values and the y predictions. Then each of those lists is stored in a series where the index is the classifier.

In [35]:
list_of_18_classifiers = ['Contact', 'Contact_E_Mail_Address', 'Contact_Phone_Number', 
                       'Identifier_Cookie_or_similar_Tech', 'Identifier_Device_ID', 'Identifier_IMEI',
                        'Identifier_MAC', 'Identifier_Mobile_Carrier',
                        'Location', 'Location_Cell_Tower', 'Location_GPS', 'Location_WiFi',
                        'SSO', 'Facebook_SSO',
                        '1st_party', '3rd_party',
                        'PERFORMED', 'NOT_PERFORMED'] # cross-checked from table on pg 4 of the paper

model_results = pd.Series(range(len(list_of_18_classifiers)),
                          index=list_of_18_classifiers, dtype=object)

Running the model and saving the results:

In [36]:
# Running the model
classifier_val_prediction = fitted_search.predict(Xval)

# saving the model results for future use
model_results[classifier] = [fitted_search, yval, classifier_val_prediction]
model_results.to_pickle("objects/model_results.pkl")

Showing confusion matrix and classification report:

In [38]:
def show_confusion_matrix(classifier_y_val, classifier_val_prediction):
    """
    Put the actual y values and the predicted y values into a dataframe with labels and display it
    """
    cf_matrix = confusion_matrix(classifier_y_val, classifier_val_prediction)
    cf_df = pd.DataFrame(
        cf_matrix, columns=["Predicted Negative", "Predicted Positive"], index=["True Negative", "True Positive"])
    display(cf_df)

In [39]:
show_confusion_matrix(yval, classifier_val_prediction)

,Predicted Negative,Predicted Positive
True Negative,1866,108
True Positive,86,591


In [41]:
print(classification_report(yval, classifier_val_prediction))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1974
           1       0.85      0.87      0.86       677

    accuracy                           0.93      2651
   macro avg       0.90      0.91      0.90      2651
weighted avg       0.93      0.93      0.93      2651



**Scoring**

Only the F1 scores of each classifier is given in the paper (Table 1 pg 4). I assume that this is F1 for class 1, and so we can use that for direct comparison.

Looks like this scored okay.  Looking at the scores on both the training and validation data, there is somewhat of a gap between the two which implies some overfitting.

Reviewing whether to focus on Positive F1 score or the negative F1 score:

"Negative F-1 score" is important because it relates to when a policy fails to mention an important practice (relevant for detecting violations, important to Story et al.).  We want to be sure that if a policy fails to mention it, the classifier correctly states that it is not mentioned.

Relatedly Negative Recall is the proportion of *When it was not in, did it say that it was not in?*<br>
Negative Precision is *When it predicted that it wasn't in, how often was that the case?*

Positive recall is *When it was in, what was the chance it was identified?*  <br>
Positive precision is *When it was predicted to be in, what was the chance that it was in?*

# Section 2: modelling pipeline

Now I can apply the same steps to every classifier to be trained.

Firstly, what are the required inputs I need to be ready before training all the classifiers?

## Requirements for modelling pipeline

1. List of all the different classifiers to train
2. Empty table of classifier results to populate
3. `df_for_pipelining_train/val/test`. This is the appropriate dataframe with all the X and y data.
4. tf-idf column transformers to use in the pipeline.

In [2]:
# repeating this list here so that the below cells can all be ran without running any above cells

# Requirement 1: List of all the different classifiers to train
list_of_18_classifiers = ['Contact', 'Contact_E_Mail_Address', 'Contact_Phone_Number', 
                       'Identifier_Cookie_or_similar_Tech', 'Identifier_Device_ID', 'Identifier_IMEI',
                        'Identifier_MAC', 'Identifier_Mobile_Carrier',
                        'Location', 'Location_Cell_Tower', 'Location_GPS', 'Location_WiFi',
                        'SSO', 'Facebook_SSO',
                        '1st_party', '3rd_party',
                        'PERFORMED', 'NOT_PERFORMED'] # cross-checked from table on pg 4 of the paper

# Requirement 2: Empty table of classifier results to populate
model_results = pd.Series(range(len(list_of_18_classifiers)),
                          index=list_of_18_classifiers, dtype=object)

full_results_table = pd.DataFrame(model_results, columns=["Frequency in train set"]).copy()

In [3]:
# Requirement 3: Loading the data

df_for_pipelining = pd.read_pickle("objects/crafted_features_df.pkl")

# Create separate dataframes for each group
df_for_pipelining_train = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'TRAINING' ].copy()
df_for_pipelining_val = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'VALIDATION' ].copy()
df_for_pipelining_test = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'TEST' ].copy()

# now that I have used the 'policy type' column for referring to train/validate/test, 
# I can delete that column along with other unneccesary columns.
for dataframe in [df_for_pipelining_train, df_for_pipelining_val, df_for_pipelining_test]:
    dataframe.drop(columns=['source_policy_number', 'policy_type', 'contains_synthetic',
           'policy_segment_id', 'annotations', 'sentences'], inplace=True)
    dataframe.reset_index(inplace=True, drop=True)

In [4]:
# Requirement 4: tf-idf column transformers to use in the pipeline

tfidf_unigrams = TfidfVectorizer(ngram_range=(1,1), stop_words='english', binary=True)
tfidf_withbigrams = TfidfVectorizer(ngram_range=(1,2), stop_words='english', binary=True)

# Create the column transformations list with columns to apply to (for the column transformer)
col_transform_unigrams = [('unigrams_only', tfidf_unigrams, 'segment_text')]
col_transform_withbigrams = [('with_bigrams', tfidf_withbigrams, 'segment_text')]

In [5]:
clean_annotation_features = pd.read_pickle("objects/clean_annotation_features.pkl")

Next, what are the required steps that will happen in training and evaluating each different classifier?

The steps are:

- Select the classifier
- Separate the training data into X and y
- Set up filter for sentence filtering to use in Pipeline
- CV Grid Search
- Separate the validation data into X and y, run the model on them and save the results

In [15]:
def full_modelling_pipeline(classifier, df_for_pipelining, df_for_evaluation, model_results_series, 
                            full_results_table, col_transform_unigrams, col_transform_withbigrams):
    
    """
    Inputs:
    
        classifier: name (exact string) of the target column to predict
        
        df_for_pipelining: Dataframe with text segments, classifiers and crafted features
        
        model_results_series:   The empty series to save the results (classifier-objects and predictions) to. 
                                A pickle file of the same name will be saved with the results. 
                                
        full_results_table:     An empty dataframe to show the results of all the classifiers 
        
        col_transform_unigrams:    for the column transformer, column transformation with column to apply to
        
        col_transform_withbigrams: as above
    """
    
    # Step 1 – declare the classifier
    
    print(f"Running for classifier: {classifier}")
    start_code_time = time.time()
    
    # Step 2 – Separate into X and y
    
    # the crafted features columns happen to be all those after and including 'contact info', so I 
    # use every column after and including the first crafted feature, which happens to be 'contact info'
    X = pd.concat(
        [df_for_pipelining['segment_text'], 
         df_for_pipelining.loc[:,'contact info':]],
        axis=1
    ).copy()

    y = df_for_pipelining[classifier]
    
    # Step 3 – filter for sentence filtering to use in Pipeline
    
    # filtering the crafted features to get the list of features from the same row that lists the classifier
    classifier_features = clean_annotation_features[ clean_annotation_features['annotation'] == classifier ]     \
                            .reset_index().at[0,'features']
    
    # true/false boolean series for sentence filtering:
    sf_filter = ((X[classifier_features] > 0)\
                     .sum(axis=1) > 0 )
    
    # true/false boolean series for balanced downsizing filter:
    positive_rows = (y == 1)
    negative_rows = (y == 0)
    balanced_downsize_filter = (
        positive_rows |
        negative_rows.where(negative_rows == True).dropna().sample(n=positive_rows.sum(), replace=False)
    )
    
    print("Ready for grid search.")
    
    # Step 4 – CV Grid Search
    
    fitted_search = model_pipeline_step_4(X, y, sf_filter, balanced_downsize_filter, 
                                          col_transform_unigrams, col_transform_withbigrams)

    print("Ready for evaluation")
    
    # Step 5 – separate test (evaluation) data into X and y, run the model on them and save the results
    
    X_eval = pd.concat(
        [df_for_evaluation['segment_text'], 
         df_for_evaluation.loc[:,'contact info':]],
        axis=1
    ).copy()

    y_eval = df_for_evaluation[classifier]
    
    # Running the model
    classifier_prediction = fitted_search.predict(X_eval)

    # saving the model results for future use
    model_results_series[classifier] = [fitted_search, y_eval, classifier_prediction]
    model_results_series.to_pickle("objects/model_results.pkl")
    
    # Populating full results table
    print("Populating results table")
    
    # Total instances in dataset
    full_results_table.loc[classifier, "Frequency in train set"] = \
        df_for_pipelining[classifier].sum()

    # Neg F1
    full_results_table.loc[classifier, "Neg F1"] = \
        f1_score(model_results_series[classifier][1].copy(), model_results_series[classifier][2].copy(), pos_label=0)

    # Pos F1
    full_results_table.loc[classifier, "Pos F1"] = \
        f1_score(model_results_series[classifier][1].copy(), model_results_series[classifier][2].copy(), pos_label=1)

    # Sentence Filtering
    full_results_table.loc[classifier, "Sentence Filtering"] = \
        str(fitted_search.best_estimator_.steps[0][1])

    # Including Bigrams or not
    full_results_table.loc[classifier, "including bigrams or not"] = \
        fitted_search.best_estimator_.named_steps["tfidf"].get_params()["transformers"][0][0]

    # Model type
    full_results_table.loc[classifier, "SVM or Logistic Regression"] = \
        fitted_search.best_estimator_.named_steps["model"]
    
    # Tests
    if type(model_results_series[classifier]) == int:
        print("Model results not saved.")
        raise NotSavedError("Check model results")
    
    print(f"The runtime for {classifier} was {round(time.time() - start_code_time, 5)}")
    print()

In [16]:
def model_pipeline_step_4(X, y, sf_filter, balanced_downsize_filter, 
                          col_transform_unigrams, col_transform_withbigrams):

    cachedir = mkdtemp() # Memory dump to help with processing

    sf_kw_args = {'df_filter': sf_filter, 'sf_filter': sf_filter, 'balanced_downsize_filter': balanced_downsize_filter}
    
    # In the pipeline_sequences list, the syntax requires that each step is named and some value is provided,
    # but that value will be changed as each different option is looped through in the parameter grid.
    pipeline_sequences = [
        ('sentence_filtering', FunctionSampler(func=priv_pol_funcs.sentence_filtering, validate=False, kw_args=sf_kw_args)),
        ('tfidf', ColumnTransformer(col_transform_withbigrams, remainder='passthrough')), 
        ('model', LogisticRegression(random_state=1, max_iter=1000))
    ]

    pipe = Pipeline(pipeline_sequences, memory = cachedir)

    param_grid = [
        {
            'model': [LogisticRegression(random_state=1, max_iter=1000)],
            'sentence_filtering': [FunctionSampler(func=priv_pol_funcs.sentence_filtering, validate=False, kw_args=sf_kw_args), 
                                   None],
            'tfidf': [ColumnTransformer(col_transform_withbigrams, remainder='passthrough'),
                      ColumnTransformer(col_transform_unigrams, remainder='passthrough')]
        },
        {
            'model': [SVC(kernel='linear', class_weight='balanced', random_state=1)],
            'model__C': [0.1, 1, 10],
            'sentence_filtering': [FunctionSampler(func=priv_pol_funcs.sentence_filtering, validate=False, kw_args=sf_kw_args), 
                                   None],
            'tfidf': [ColumnTransformer(col_transform_withbigrams, remainder='passthrough'),
                      ColumnTransformer(col_transform_unigrams, remainder='passthrough')]
        }
    ]

    # Create grid search object
    grid_search_object = GridSearchCV(estimator=pipe, param_grid = param_grid, cv = 5, verbose=1, n_jobs=-1, scoring='f1')
    
    fitted_search = grid_search_object.fit(X, y)
    
    return fitted_search 

Pass every different classifier into the modelling pipeline:

**! This takes a few minutes to run**

In [17]:
initial_results = model_results.copy()

In [18]:
for each_classifier in list_of_18_classifiers:
    
    full_modelling_pipeline(classifier=each_classifier, df_for_pipelining=df_for_pipelining_train, 
    df_for_evaluation=df_for_pipelining_val, model_results_series=initial_results, 
    full_results_table = full_results_table, 
    col_transform_unigrams=col_transform_unigrams, col_transform_withbigrams=col_transform_withbigrams)

Running for classifier: Contact
Ready for grid search.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.14s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.10s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib'

Ready for evaluation
Populating results table
The runtime for Contact was 27.97552

Running for classifier: Contact_E_Mail_Address
Ready for grid search.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.37s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.36s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib'

Ready for evaluation
Populating results table
The runtime for Contact_E_Mail_Address was 35.71502

Running for classifier: Contact_Phone_Number
Ready for grid search.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

Ready for evaluation
Populating results table
The runtime for 3rd_party was 54.97772

Running for classifier: PERFORMED
Ready for grid search.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.08s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 1.12s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib'

Ready for evaluation
Populating results table
The runtime for PERFORMED was 94.30847

Running for classifier: NOT_PERFORMED
Ready for grid search.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 0.70s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/Users/chinchcliffe/opt/anaconda3/envs/priv_pol_nlp/lib/python3.10/site-packages/imblearn/pipeline.py:240: UserWarning: Persisting input arguments took 0.73s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib'

Ready for evaluation
Populating results table
The runtime for NOT_PERFORMED was 36.58304



In [19]:
full_results_table

,Frequency in train set,Neg F1,Pos F1,Sentence Filtering,including bigrams or not,SVM or Logistic Regression
Contact,128,0.992875,0.660550,None,with_bigrams,"SVC(C=0.1, class_weight='balanced', kernel='li..."
Contact_E_Mail_Address,662,0.986949,0.839196,None,unigrams_only,"LogisticRegression(max_iter=1000, random_state=1)"
Contact_Phone_Number,346,0.991876,0.839216,None,with_bigrams,"SVC(C=1, class_weight='balanced', kernel='line..."
Identifier_Cookie_or_similar_Tech,596,0.990924,0.903084,None,with_bigrams,"SVC(C=1, class_weight='balanced', kernel='line..."
Identifier_Device_ID,332,0.994713,0.861538,None,with_bigrams,"SVC(C=10, class_weight='balanced', kernel='lin..."
Identifier_IMEI,50,0.999620,0.937500,None,with_bigrams,"SVC(C=0.1, class_weight='balanced', kernel='li..."
Identifier_MAC,85,0.998669,0.837209,None,unigrams_only,"SVC(C=1, class_weight='balanced', kernel='line..."
Identifier_Mobile_Carrier,60,0.996014,0.363636,None,with_bigrams,"SVC(C=1, class_weight='balanced', kernel='line..."
Location,537,0.986813,0.825737,None,unigrams_only,"LogisticRegression(max_iter=1000, random_state=1)"
Location_Cell_Tower,89,0.995615,0.596491,None,with_bigrams,"SVC(C=1, class_weight='balanced', kernel='line..."


In [20]:
print(f"Negative F1 mean: {full_results_table['Neg F1'].mean()}")
print(f"Positive F1 mean: {full_results_table['Pos F1'].mean()}")

Negative F1 mean: 0.9872217712600695
Positive F1 mean: 0.7773976302357587


In [37]:
initial_results['Identifier_Device_ID'][0].best_estimator_

Pipeline(memory='/var/folders/d2/s5sb3p416xbgzjf589bgpp_w0000gn/T/tmpj2c7b1ym',
         steps=[('sentence_filtering', None),
                ('tfidf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('with_bigrams',
                                                  TfidfVectorizer(binary=True,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'segment_text')])),
                ('model',
                 SVC(C=10, class_weight='balanced', kernel='linear',
                     random_state=1))])

The results for any classifier can then be retrieved from the results series. The series stores the model and the actual and predicted y-values in the form `[model_object, true_y, predicted_y]`

Here is how a classification report could be produced for one of the classifiers.

In [38]:
print(classification_report(initial_results['Contact_E_Mail_Address'][1] , 
                            initial_results['Contact_E_Mail_Address'][2]))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2429
           1       0.95      0.75      0.84       222

    accuracy                           0.98      2651
   macro avg       0.96      0.87      0.91      2651
weighted avg       0.98      0.98      0.97      2651



# Old from here:

Another next step is to move the pipeline functions I have created to a separate file to tidy up the notebook. For now, please accept the following sequence of custom functions. They mimic the steps already performed above for the 1st Party classifier.

The first function calls the different sub-functions.

In [37]:
def full_modelling_pipeline(classifier, model_results_series, sentence_filtering=True, inspect_flow=False):
    
    """
    Inputs:
    
        classifier: name (exact string) of the target column to predict
        
        model_results_series:   the empty series to save the results to. 
                                A pickle file of the same name will be saved with the results. 
        
        sentence_filtering:     sentence_filtering takes a boolean, default True. 
                                If False, the flow will ommit the sentence filtering step.
        
        inspect_flow: Passing inspect_flow=True will print out the shape of dataframes moving through the flow 
    """
#     if sentence_filtering == False:
#         raise NotImplementedError
    
    # Step 1 – declare the classifier
    
    print(f"Running for classifier: {classifier}")
    start_code_time = time.time()
    
    # Step 2 – execute sentence filtering on the data to use
    
    clean_annotation_features = pd.read_pickle("objects/clean_annotation_features.pkl")
    
    if sentence_filtering == True:
        df_for_pipelining_train_SF = model_pipeline_step_2(classifier, clean_annotation_features)
    elif sentence_filtering == False:
        df_for_pipelining_train_SF = df_for_pipelining_train.copy()
        df_for_pipelining_train_SF.reset_index(inplace=True, drop=True)
    else:
        raise Sentence_Filtering_Not_Boolean_Error
    
    if inspect_flow == True: 
        print(f"Shape of {classifier} train df after sentence filtering is: {df_for_pipelining_train_SF.shape}")
    
    # Step 3 – create the X and y data
    
    classifier_X_train, tfidfTransformer = model_pipeline_step_3_1(df_for_pipelining_train_SF)
    
    classifier_y_train = model_pipeline_step_3_2(classifier, df_for_pipelining_train_SF)
    
    if inspect_flow == True: 
        print(f"classifier_X_train (made of CFs plus tf-idf matrix): {classifier_X_train.shape}")
        print(f"classifier_y_train: {classifier_y_train.shape}")
        print("classifier_y_train value counts:")
        print(classifier_y_train.value_counts())
    
    # Step 4 – run CV Grid Search
    
    fitted_search = model_pipeline_step_4(classifier_X_train, classifier_y_train)
    
    # Step 5 – run the best model on the validation set and save the results
    
    classifier_X_val, classifier_y_val = model_pipeline_step_5_1(classifier, df_for_pipelining_val, tfidfTransformer)
    if inspect_flow == True: 
        print(f"classifier_X_val: {classifier_X_val.shape}")
        print(f"classifier_y_val: {classifier_y_val.shape}")
    
    model_pipeline_step_5_2(classifier, fitted_search, classifier_X_val, classifier_y_val, model_results_series)
    
    # Tests
    if type(model_results_series[classifier]) == int:
        print("Model results not saved.")
        raise NotSavedError("Check model results")
    
    print(f"The runtime for {classifier} was {round(time.time() - start_code_time, 5)}")
    print()

In [38]:
def model_pipeline_step_2(classifier, clean_annotation_features):
    """
    Step 2 – Get crafted features for classifier to use for sentence filtering.
    
    Inputs: classifier name, annotation features
    
    Outputs: The train dataframe (containing the segment text, classifiers and features)
        with rows filtered out that don't contain segments featuring a related feature ("sentence filtering applied").
    """
    
    # Filtering the annotations & features table to get the list object from the same row that lists the classifier:
    classifier_features = clean_annotation_features[ clean_annotation_features['annotation'] == classifier ] \
                            .reset_index().at[0,'features']
    
    # Filter the DF for rows where any of those features is 1:
    df_for_pipelining_train_SF = df_for_pipelining_train[
        ( (df_for_pipelining_train[classifier_features] > 0)
         .sum(axis=1) > 0 )]
    df_for_pipelining_train_SF.reset_index(inplace=True, drop=True)
    
    return df_for_pipelining_train_SF

In [39]:
def model_pipeline_step_3_1(df_for_pipelining_train_SF):
    """
    Step 3.1 – get the training data X values
    
    Inputs:  The train dataframe with sentence filtering applied
    
    Outputs: 
        - The training data X values to be passed into the models (crafted features & tf-idf matrix)
        - The tf-idf transformer
    """
    
    # Separate into X
    
    tfidfTransformer = TfidfVectorizer(ngram_range=(1,2), stop_words='english', binary=True)

    train_tfidf = tfidfTransformer.fit_transform(df_for_pipelining_train_SF['segment_text'])
    
    # Extract crafted features columns:
    
    classifier_X_train_cfs = df_for_pipelining_train_SF.loc[:,'contact info':].copy() 
        # Use every column after and including the first crafted feature, which happens to be 'contact info'
    
    if classifier_X_train_cfs.shape[1] != 476:
        print(f"Should be left with the 476 crafted features (CF). CF shape is: {classifier_X_train_cfs.shape}")
        raise Step_3_CF_error("Crafted features not being applied correctly")

    # Convert to sparse
    classifier_X_train_cfs = csr_matrix(classifier_X_train_cfs)

    # Combine CF columns with TF-IDF to create X
    classifier_X_train = hstack([classifier_X_train_cfs, train_tfidf ])
    
    return classifier_X_train, tfidfTransformer

In [40]:
def model_pipeline_step_3_2(classifier, df_for_pipelining_train_SF):
    """
    Step 3.2 – Get the training data y values
    
    Inputs:
        - Classifier name
        - The train dataframe with sentence filtering applied
    
    Outpus:
        The dataframe filtered to only the column with the classifier (the y values to be passed into the models).
    """
    
    # Separate into y
    
    classifier_y_train = df_for_pipelining_train_SF.loc[:,classifier].copy()
    
    if classifier_y_train.max() != 1:
        print(f"Highest value should be one. Highest value is: {classifier_y_train.max()}")
        raise Step_3_y_error("train target colum not binary")
    
    return classifier_y_train

In [41]:
def model_pipeline_step_4(classifier_X_train, classifier_y_train):
    """
    Step 4 – Run a CV Grid Search
    
    Inputs: 
        - The X data to be passed into the models
        - The y data to be passed into the models
    
    Outputs: A fitted search object corresponding to the best model for the classifier
    """
    
    # Grid Search
    
    cachedir = mkdtemp() # Memory dump to help with processing

    pipeline_sequences = [
            ('SVC', SVC(kernel='linear', class_weight='balanced', random_state=1)) ]
    pipe = Pipeline(pipeline_sequences, memory = cachedir)

    svc_params = {'SVC__C': [0.1, 1, 10]}

    # Create grid search object
    grid_search_object = GridSearchCV(estimator=pipe, param_grid = svc_params, cv = 5, verbose=0, n_jobs=-1, scoring='f1')
    
    fitted_search = grid_search_object.fit(classifier_X_train, classifier_y_train)
    
    return fitted_search

In [42]:
def model_pipeline_step_5_1(classifier, df_for_pipelining_val, tfidfTransformer):
    """
    Step 5.1 – Prepare the validation data
    
    Inputs:
        - The classifier name
        - The validation dataframe
        - The tf-idf transformer that was fit to the training data
    
    Ouptuts:
        - The X data (from the validation dataset) to run the model against
        - The y data (from the validation dataset) to run the model against
    """
    
    # create validate X and y

    val_tfidf = tfidfTransformer.transform(df_for_pipelining_val['segment_text'])
    
    # Extract CF columns:
    classifier_X_val_cfs = df_for_pipelining_val.loc[:,'contact info':].copy()
    
    # convert to sparse:
    classifier_X_val_cfs = csr_matrix(classifier_X_val_cfs)

    # combine CF columns with TF-IDF to create X:
    classifier_X_val = hstack([classifier_X_val_cfs, val_tfidf ])

    classifier_y_val = df_for_pipelining_val.loc[:,classifier].copy()

    # Ensure Y_val only has binary values
    if classifier_y_val.isin([0, 1]).all().all() != True:
        raise Step_5_val_error("Validation target column not binary")
    
    return classifier_X_val, classifier_y_val

In [43]:
def model_pipeline_step_5_2(classifier, fitted_search, classifier_X_val, classifier_y_val, model_results_series):
    """
    Step 5.2 – Put the model through the validation data to get results for this classifier.
    
    Inputs:
        - Classifier name
        - The fitted search object corresponding to the best model for the classifier
        - The X data (from the validation dataset) to run the model against
        - The y data (from the validation dataset) to run the model against
        - A series indexed by classifier name to store the model results to.
    
    Outputs: None. 
    
    Actions: Saves the results to the model_results_series. It saves a list with 3 values:
        - the fitted search object
        - the y values in the validation dataset
        - the y values predicted by the model    
    """
    
    # scoring
    classifier_val_prediction = fitted_search.predict(classifier_X_val)

    # save the model to a series
    model_results_series[classifier] = [fitted_search, classifier_y_val, classifier_val_prediction]
    
    # save the series to disk to prevent data loss in case of crash
    model_results_series.to_pickle("objects/most_recent_model_results.pkl")

# Run all classifiers through the pipeline

Creating a series to store the results:

In [12]:
initial_results_sf = pd.Series(range(len(list_of_18_classifiers)),
                          index=list_of_18_classifiers, dtype=object)

Pass every different classifier into the modelling pipeline:

**! This takes a few minutes to run**

In [69]:
for each_classifier in list_of_18_classifiers:
    
    full_modelling_pipeline(each_classifier, model_results_series=initial_results_sf, 
                            sentence_filtering=False, inspect_flow=True)

Running for classifier: Contact
Shape of Contact train df after sentence filtering is: (8068, 511)
classifier_X_train (made of CFs plus tf-idf matrix): (8068, 113262)
classifier_y_train: (8068,)
classifier_y_train value counts:
0    7940
1     128
Name: Contact, dtype: int64
classifier_X_val: (2651, 113262)
classifier_y_val: (2651,)
The runtime for Contact was 9.63536

Running for classifier: Contact_E_Mail_Address
Shape of Contact_E_Mail_Address train df after sentence filtering is: (8068, 511)
classifier_X_train (made of CFs plus tf-idf matrix): (8068, 113262)
classifier_y_train: (8068,)
classifier_y_train value counts:
0    7406
1     662
Name: Contact_E_Mail_Address, dtype: int64
classifier_X_val: (2651, 113262)
classifier_y_val: (2651,)
The runtime for Contact_E_Mail_Address was 25.06858

Running for classifier: Contact_Phone_Number
Shape of Contact_Phone_Number train df after sentence filtering is: (8068, 511)
classifier_X_train (made of CFs plus tf-idf matrix): (8068, 113262)
cl

The results for any classifier can then be retrieved from the results series. The series stores the actual and predicted y-values in the form `[model_object, true_y, predicted_y]`

Here is an example classification report for one of the classifiers.

In [53]:
print(classification_report(initial_results_sf['Contact_E_Mail_Address'][1] , initial_results_sf['Contact_E_Mail_Address'][2]))


              precision    recall  f1-score   support

           0       0.87      0.55      0.67      1974
           1       0.36      0.75      0.49       677

    accuracy                           0.60      2651
   macro avg       0.61      0.65      0.58      2651
weighted avg       0.74      0.60      0.62      2651



# Creating model evaluation table

In [70]:
initial_model_results = pd.read_pickle("objects/most_recent_model_results.pkl")

I will display the results for all classifiers in a dataframe:

In [71]:
# take the classifiers as the index for our model results df
initial_f1s = pd.DataFrame(initial_model_results, columns=["Neg F1"]).copy()

# then populate the dataframe with F1 scores by reference to the 'initial_model_results' series.
# the initial_model_results series stores the actual and predicted y-values in the form [model_object, true_y, predicted_y]
for index in initial_f1s.index:
    initial_f1s.loc[index, "Neg F1"] = \
        f1_score(initial_model_results[index][1].copy(), initial_model_results[index][2].copy(), pos_label=0)
    initial_f1s.loc[index, "Pos F1"] = \
        f1_score(initial_model_results[index][1].copy(), initial_model_results[index][2].copy(), pos_label=1)

In [73]:
initial_f1s

,Neg F1,Pos F1
Contact,0.992875,0.660550
Contact_E_Mail_Address,0.986464,0.845070
Contact_Phone_Number,0.991876,0.839216
Identifier_Cookie_or_similar_Tech,0.990924,0.903084
Identifier_Device_ID,0.993328,0.834951
Identifier_IMEI,0.99962,0.937500
Identifier_MAC,0.998289,0.790698
Identifier_Mobile_Carrier,0.996014,0.363636
Location,0.98595,0.823529
Location_Cell_Tower,0.995615,0.596491


In [72]:
print(f"Negative F1 mean: {initial_f1s['Neg F1'].mean()}")
print(f"Positive F1 mean: {initial_f1s['Pos F1'].mean()}")

Negative F1 mean: 0.98712093292967
Positive F1 mean: 0.7743867067641877


Many things to discuss based on the above:
- The two lowest scores
- A wide range of scores
- Neg F1 tends to be higher than Pos F1
- Compare with results from the paper
- Equal results for the two SSO classifiers


# Discussion of low F1 scores
There is one zero for negative F1 `Identifier_IMEI` Let's start by investigating `Identifier_IMEI`. 

A segment is annotated with 'Identifier_IMEI' if the segment describes how the company uses IMEI data to identify a customer. IMEI (International Mobile Equipment Identity) is a unique identification number all phone devices have, and can be used to track the history of the handset (including checking whether the phone has ever been reported as stolen).  First let's look at the confusion matrix. 

In [58]:
show_confusion_matrix(initial_model_results['Identifier_IMEI'][1].copy(), initial_model_results['Identifier_IMEI'][2].copy())

,Predicted Negative,Predicted Positive
True Negative,2,1972
True Positive,4,673


The classifier almost never predicted that this label is not present.  This is likely because in the subset of the data that the classifier was trained on, all of the observations were positive cases. This will be the result of the sentence filtering – the classifier was only trained using text segments that mentioned phrases related to IMEI, all of which happened to be annotated as indicating that the practice was described.

This could mean that a rule-based classifier would be suitable for classifying the IMEI practice.  For a model-based classifier, I would need to use a wider range of features for sentence filtering, or would not conduct sentence filtering.

We can see the features used:

In [59]:
clean_annotation_features[clean_annotation_features['annotation']=='Identifier_IMEI']['features'].values

array([list(['imei', 'international mobile equipment', 'equipment id'])],
      dtype=object)

As expected there are very few related phrases, all of which are somewhat technical.

Another implication could be that because IMEI is so specific, companies will not mention anything related to it in their privacy policies unless they collect this data.

**Location Cell Tower**

I think a similar problem has occurred here. Let's check.

In [60]:
show_confusion_matrix(initial_model_results['Location_Cell_Tower'][1].copy(), initial_model_results['Location_Cell_Tower'][2].copy())

,Predicted Negative,Predicted Positive
True Negative,142,1832
True Positive,31,646


In [61]:
clean_annotation_features[clean_annotation_features['annotation']=='Location_Cell_Tower']['features'].values

array([list(['base station', 'cell tower', 'cell id', 'network-based', 'network based', 'cellular network location', 'approximate location', 'approximate geo', 'approximate device location', 'approximate device geo', 'approximate a location', 'coarse location', 'coarse geo', 'coarse device location', 'coarse device geo', 'rough location', 'rough geo', 'rough device location', 'rough device geo', 'imprecise location', 'imprecise geo', 'imprecise device location', 'imprecise device geo', 'located with less precision', 'non-precise location', 'non-precise geo', 'non-precise device location', 'non-precise device geo', 'non-specific location', 'non-specific geo', 'non-specific device location', 'not-specific device geo', 'not-specific location', 'not-specific geo', 'not-specific device location', 'not-specific device geo', 'general location', 'general geo', 'general device location', 'general device geo', 'generally where you are located', 'based on proximity'])],
      dtype=object)

It looks like there is a similar problem with most of the training data probably having been positive cases, but the cause is less clear because there are a large number of different related crafted features.  By recollection from manually inspecting the frequencies of different crafted features, most of those related to location had surprisingly low frequency in the text because policies all tended to use some exact phrasing for location. As we can tell, the above crafted features are all around imprecise location.

**Training this many classifiers**

Given the large number of different classifiers being trained, there is a chance that at least one of them is poorly represented in the training set by chance of the random split, or other random factors in the validation data.

## A wide range of scores

With training this many classifiers a wide range of scores is expected, but the targets with the most data (the parties and modalities) have the best results.

## Neg F1 tends to be higher than Pos F1

This is partially likely due to imbalance in the data, which sentence filtering was supposed to help with.

## Compare with results from the paper

My results are not particularly similar to those listed in the table on page 4.  Most notably the baseline models for Story et al. hardly ended up with any classifiers with low scores, so it's possible that the different ways that they manipulated the data had a significant effect.  The main differences were where in the process they applied tf-idf and how they did sentence filtering. 

## Matching scores for SSO and FacebookSSO

The SSO ("Single Sign On") annotation is applied to a segment if it discusses what the company does with the data used to facilitate a single sign on service, such as when you sign into an app through your google or facebook account.  This is always passed to the appropriate third party to enact the service. The scores for both "SSO" and "Facebook SSO" are the same because the crafted features and most of the annotations are the same.  Again let's start by looking at the confusion matrix for each.

In [62]:
show_confusion_matrix(initial_model_results['SSO'][1].copy(), initial_model_results['SSO'][2].copy())

,Predicted Negative,Predicted Positive
True Negative,1967,7
True Positive,592,85


In [63]:
show_confusion_matrix(initial_model_results['Facebook_SSO'][1].copy(), initial_model_results['Facebook_SSO'][2].copy())

,Predicted Negative,Predicted Positive
True Negative,1967,7
True Positive,592,85


The proportions of both these annotations must be the same in the validation set. I think this could be due to an issue with my data manipulation as I expect there to be some difference.

This time the model very rarely predicted that a segment contained a practice relating to SSO, most likely because there is such great class imbalance in the training data (not the validation data) even after applying sentence filtering. This has caused our model to be biased.

In [64]:
df_for_pipelining_train['SSO'].value_counts()

0    7922
1     146
Name: SSO, dtype: int64

Yes, the proportions are hugely different in the training data and very imbalanced. For this classifier it definitely looks like we got unlucky with the train/validate split (assuming it was random).

# Final model performance on test set

Now I will train all classifiers using the Train and Validate data sets combined and test the score on the test set.

In [65]:
final_results = pd.Series(range(len(list_of_18_classifiers)),
                          index=list_of_18_classifiers, dtype=object)

I am yet to refactor the code to smoothly adapt my functions to work on the test set so I will have to re-assign the targets of the input variables to the pipeline:

In [66]:
df_for_pipelining = pd.read_pickle("objects/crafted_features_df.pkl")

df_for_pipelining_train = df_for_pipelining.loc[(df_for_pipelining['policy_type'] == 'TRAINING') | (df_for_pipelining['policy_type'] == 'VALIDATION')].copy()
df_for_pipelining_test = df_for_pipelining.loc[df_for_pipelining['policy_type'] == 'TEST' ].copy()
df_for_pipelining_test.reset_index(inplace=True, drop=True)

# now that I have used the 'policy type' column for referring to train/validate/test, 
# I can delete that column along with other unneccesary columns.
for dataframe in [df_for_pipelining_train, df_for_pipelining_test]:
    dataframe.drop(columns=['source_policy_number', 'policy_type', 'contains_synthetic',
           'policy_segment_id', 'annotations', 'sentences'], inplace=True)

df_for_pipelining_val = df_for_pipelining_test

Running modelling pipeline for all classifiers

**! This could take some time to run:**

In [67]:
for each_classifier in list_of_18_classifiers:
    full_modelling_pipeline(each_classifier, model_results_series=final_results, 
                            sentence_filtering=True, inspect_flow=True)

Running for classifier: Contact
Shape of Contact train df after sentence filtering is: (480, 511)
df_for_pipelining_train_SF: (480, 511)
classifier_X_train (made of CFs plus tf-idf matrix): (480, 16471)
classifier_y_train: (480,)
classifier_X_val: (4824, 16471)
classifier_y_val: (4824,)
The runtime for Contact was 1.25053

Running for classifier: Contact_E_Mail_Address
Shape of Contact_E_Mail_Address train df after sentence filtering is: (731, 511)
df_for_pipelining_train_SF: (731, 511)
classifier_X_train (made of CFs plus tf-idf matrix): (731, 20188)
classifier_y_train: (731,)
classifier_X_val: (4824, 20188)
classifier_y_val: (4824,)
The runtime for Contact_E_Mail_Address was 1.54034

Running for classifier: Contact_Phone_Number
Shape of Contact_Phone_Number train df after sentence filtering is: (714, 511)
df_for_pipelining_train_SF: (714, 511)
classifier_X_train (made of CFs plus tf-idf matrix): (714, 22018)
classifier_y_train: (714,)
classifier_X_val: (4824, 22018)
classifier_y_val:

In [68]:
final_model_results = pd.read_pickle("objects/most_recent_model_results.pkl")

In [69]:
# take the classifiers as the index for our model results df
final_f1s = pd.DataFrame(final_model_results, columns=["Neg F1"]).copy()

# then populate the dataframe with F1 scores by reference to the 'initial_model_results' series.
# the initial_model_results series stores the actual and predicted y-values in the form [model_object, true_y, predicted_y]
for index in final_f1s.index:
    final_f1s.loc[index, "Neg F1"] = f1_score(final_model_results[index][1].copy(), final_model_results[index][2].copy(), pos_label=0)
    final_f1s.loc[index, "Pos F1"] = f1_score(final_model_results[index][1].copy(), final_model_results[index][2].copy(), pos_label=1)
display(final_f1s)
print(f"Negative F1 mean: {final_f1s['Neg F1'].mean()}")
print(f"Positive F1 mean: {final_f1s['Pos F1'].mean()}")

,Neg F1,Pos F1
Contact,0.91178,0.633547
Contact_E_Mail_Address,0.386906,0.320734
Contact_Phone_Number,0.81761,0.518854
Identifier_Cookie_or_similar_Tech,0.728218,0.427560
Identifier_Device_ID,0.566071,0.371892
Identifier_IMEI,0.004063,0.313135
Identifier_MAC,0.915175,0.556488
Identifier_Mobile_Carrier,0.472029,0.353999
Location,0.741646,0.475503
Location_Cell_Tower,0.535186,0.372625


Negative F1 mean: 0.7202505215060437
Positive F1 mean: 0.4993427790530155


Interestingly the score for Contact_E_Mail_Address has gone down significantly. It would be very interesting to explore why this might be.

The scores for SSO have only increased by a small amount despite the class imbalance issue potentially being solved by using the train and validate data.

The mean F1 scores have remained largely the same and are still much lower for Positive F1.  I feel that this is still due to such imbalance across the dataset but would like to explore more.

One conclusion is that by increasing the amount of data being used, the model performance has not really increased, meaning that the limiting factors are probably in the model creation.

# Further work

A next step is to implement an ability of the modelling pipeline to remove sentence filtering from the flow.  It is causing some issues in my model scores and Story et al. found that some classifiers' performance improves without it.

I would then work on other preprocessing steps:
- I have not applied scaling to the data. Every column is between 0 and 1 anyway so the effect would be small but it could have big effect for some classifiers.
- tf-idf: 
    - I would explore training classifiers without bigrams as Story et al. also explored this and found it to be helpful for a small number of classifiers 
    - Fixing data leak: currently, the tf-idf transformer is fitted and applied to the entire train set before cross-validation.

This leads on to a more general issue that I have created a series of functions but none of my own classes, so I would refactor into a more object-oriented framework.

Other types of models:
- Naive bayes classifiers and ensemble models such as random forest have also shown to perform well in the NLP domain and I would wish to see the performance of these models too.
- Neural Network architecture is also likely to get very strong performance, although may require more investment and training resources.

Deeper exploration into the effect of different models in this domain

- It would be insightful to explore why different preprocessing steps or why different models have different effects on different classifiers when trained on this dataset

**Thanks for reading!**

---

---

# OLD: TF-IDF matrix (old content)
The input data to the model, X, requires a union of the Crafted Features columns and the TF-IDF matrix.

Create TF-IDF matrix, fit it to the text data, and use it to transform it to create the matrix:

In [8]:
# instantiate transformer object
tfidfTransformer = TfidfVectorizer(ngram_range=(1,2), stop_words='english', binary=True)

# fit the transformer and transform the data
train_tfidf = tfidfTransformer.fit_transform(df_for_pipelining_train_SF['segment_text'])

This tf-idf matrix is stored in a sparse matrix format.

Extract crafted features columns from X and convert it to sparse format so that it can be combined with TF-IDF:

In [9]:
# Extract CF columns:
classifier_X_train_cfs = df_for_pipelining_train_SF.loc[:,'contact info':].copy() 
# the CF columns happen to be all those after and including 'contact info', so I 
# use every column after and including the first crafted feature, which happens to be 'contact info'
print(f"Should be left with the 476 different crafted features (CFs). CF shape is: {classifier_X_train_cfs.shape}")

#convert to sparse
classifier_X_train_cfs = csr_matrix(classifier_X_train_cfs)

# combine CF columns with TF-IDF to create X
classifier_X_train = hstack([classifier_X_train_cfs, train_tfidf]) 
    # hstack combines the matricies horizontally (more columns, same number of rows)

Should be left with the 476 different crafted features (CFs). CF shape is: (7297, 476)


## Create y (old content)

In [10]:
classifier_y_train = df_for_pipelining_train_SF.loc[:,classifier].copy()

Now we have our input data and target ready to pass into our model.

# Step 2: apply Sentence Filtering (old content)

Retrieving the crafted features for 1st Party to use for sentence filtering:

In [6]:
# filtering the table to get the list object from the same row that lists the classifier
classifier_features = clean_annotation_features[ clean_annotation_features['annotation'] == classifier ]     \
                        .reset_index().at[0,'features']

classifier_features

['we', 'you', 'us', 'our', 'the app', 'the software']

Filter the dataframe for rows where any of those features is 1:

In [7]:
df_for_pipelining_train_SF = df_for_pipelining_train[
    ((df_for_pipelining_train[classifier_features] > 0)
    .sum(axis=1) > 0 )]
df_for_pipelining_train_SF.reset_index(inplace=True, drop=True)

print(f"The original dataframe's shape was {df_for_pipelining_train.shape}")
print(f"The dataframe after sentence filtering is {df_for_pipelining_train_SF.shape}")

The original dataframe's shape was (8068, 511)
The dataframe after sentence filtering is (7297, 511)


We have lost a big proportion of data but still have enough.

# Evaluation on Validation dataset (Old content)

I conduct similar preprocessing steps to prepare the validate set for prediction. I can ommit sentence filtering because it is only important for training the model, and the model is already trained.

(Load the validation dataset)

Create the tfid matrix by transforming the text data (using the transformer fit on the training data), combining this with the created features, and also extracting the target y: 

In [41]:
val_tfidf = tfidfTransformer.transform(df_for_pipelining_val['segment_text'])

# Extract CF columns:
classifier_X_val_cfs = df_for_pipelining_val.loc[:,'contact info':].copy()
# convert to sparse
classifier_X_val_cfs = csr_matrix(classifier_X_val_cfs)

# combine CF columns with TF-IDF to create X
classifier_X_val = hstack([classifier_X_val_cfs, val_tfidf ])

# extracting the target y:
classifier_y_val = df_for_pipelining_val.loc[:,classifier].copy()

Running the model.

**Storing the model results** 

I create a series to store the results for each classifier I could make.

For each model, I will create a list with 3 values: the fitted search object, the y values and the y predictions. Then each of those lists is stored in a series where the index is the classifier.

In [42]:
list_of_18_classifiers = ['Contact', 'Contact_E_Mail_Address', 'Contact_Phone_Number', 
                       'Identifier_Cookie_or_similar_Tech', 'Identifier_Device_ID', 'Identifier_IMEI',
                        'Identifier_MAC', 'Identifier_Mobile_Carrier',
                        'Location', 'Location_Cell_Tower', 'Location_GPS', 'Location_WiFi',
                        'SSO', 'Facebook_SSO',
                        '1st_party', '3rd_party',
                        'PERFORMED', 'NOT_PERFORMED'] # cross-checked from table on pg 4 of the paper

model_results = pd.Series(range(len(list_of_18_classifiers)),
                          index=list_of_18_classifiers, dtype=object)

Running the model and saving the results:

In [43]:
# Running the model
classifier_val_prediction = fitted_search.predict(classifier_X_val)

# saving the model results for future use
model_results[classifier] = [fitted_search, classifier_y_val, classifier_val_prediction]
model_results.to_pickle("objects/model_results.pkl")

Showing confusion matrix and classification report:

In [55]:
def show_confusion_matrix(classifier_y_val, classifier_val_prediction):
    """
    Put the actual y values and the predicted y values into a dataframe with labels and display it
    """
    cf_matrix = confusion_matrix(classifier_y_val, classifier_val_prediction)
    cf_df = pd.DataFrame(
        cf_matrix, columns=["Predicted Negative", "Predicted Positive"], index=["True Negative", "True Positive"])
    display(cf_df)

In [54]:
show_confusion_matrix(classifier_y_val, classifier_val_prediction)

,Predicted Negative,Predicted Positive
True Negative,1865,109
True Positive,85,592


In [45]:
print(classification_report(classifier_y_val, classifier_val_prediction))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1974
           1       0.84      0.87      0.86       677

    accuracy                           0.93      2651
   macro avg       0.90      0.91      0.90      2651
weighted avg       0.93      0.93      0.93      2651

